<a href="https://colab.research.google.com/github/ilmechaJu/-Base-Line-CNN/blob/main/ch04_%EC%95%99%EC%83%81%EB%B8%94_%EA%B8%B0%EB%B2%95_%EC%88%AB%EC%9E%90%EC%86%90%EA%B8%80%EC%94%A8_%EB%B6%84%EB%A5%98_%EA%B0%9C%EC%84%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 18132736622439474593, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 7250706432
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 16001322133683374910
 physical_device_desc: "device: 0, name: Tesla P4, pci bus id: 0000:00:04.0, compute capability: 6.1"]

In [ ]:
import numpy as np

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from keras.datasets import mnist
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras import optimizers

(X_train, y_train), (X_test, y_test) = mnist.load_data()

# reshaping X data: (n, 28, 28) => (n, 784)
X_train = X_train.reshape((X_train.shape[0], -1))
X_test = X_test.reshape((X_test.shape[0], -1))

# 학습 과정을 단축시키기 위해 학습 데이터의 1/3만 활용한다
X_train, _ , y_train, _ = train_test_split(X_train, y_train, test_size = 0.67, random_state = 7)

# 타겟 변수를 one-hot encoding 한다
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

y_train = np.argmax(y_train, axis = 1)
y_test = np.argmax(y_test, axis = 1)


def mlp_model():
    model = Sequential()
    model.add(Dense(50, input_shape = (784, )))
    model.add(Activation('sigmoid'))    
    model.add(Dense(50))
    model.add(Activation('sigmoid'))    
    model.add(Dense(50))
    model.add(Activation('sigmoid'))    
    model.add(Dense(50))
    model.add(Activation('sigmoid'))    
    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    sgd = optimizers.SGD(learning_rate = 0.001)
    model.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    return model



model1 = KerasClassifier(build_fn = mlp_model, epochs = 10, verbose = 0)
model2 = KerasClassifier(build_fn = mlp_model, epochs = 10, verbose = 0)
model3 = KerasClassifier(build_fn = mlp_model, epochs = 10, verbose = 0)
model1._estimator_type = "classifier"
model2._estimator_type = "classifier"
model3._estimator_type = "classifier"


ensemble_clf = VotingClassifier(estimators = [('model1', model1), ('model2', model2), ('model3', model3)]
                                , voting = 'soft')


ensemble_clf.fit(X_train, y_train)


y_pred = ensemble_clf.predict(X_test)

print('Test accuracy:', accuracy_score(y_pred, y_test))

#  푼 답 저장하기
prob_pred = ensemble_clf.predict(X_test)
np.savetxt('y_pred.csv', prob_pred, fmt='%d')

(19800, 784) (10000, 784) (19800, 10) (10000, 10)
Test accuracy: 0.1135
